### Prepare data set for hospital benchmarking using data from the National Inpatient Sample (NIS), Healthcare Cost and Utilization Project (HCUP), Agency for Healthcare Research and Quality. This notebook is informational only, in accordance with HCUP's data use standards.

In [1]:
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:99% !important; }</style>"))
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_pickle('hcupData.pkl')

In [3]:
cases = df[['HOSP_NIS']]
cases['Cases'] = 1
hospitals = cases.groupby('HOSP_NIS').sum().reset_index().sort_values('Cases',ascending=False)
hospitals = hospitals.loc[hospitals['Cases']>100]
caseCount = hospitals[['HOSP_NIS','Cases']]
hospitalList = hospitals['HOSP_NIS'].tolist()

In [4]:
hospitalDf = df.loc[df['HOSP_NIS'].isin(hospitalList)]
hospitalDf = hospitalDf.sort_values('HOSP_NIS')

In [5]:
transfers = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['TRAN_IN']).reset_index()[['HOSP_NIS',1]]
transfers.columns = ['HOSP_NIS','TransfersIn']

In [6]:
ed = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['HCUP_ED']).reset_index()[['HOSP_NIS',1,2,3,4]]
ed['EmergencyAdmit'] = ed.iloc[:, 1:].sum(axis=1)
ed = ed[['HOSP_NIS','EmergencyAdmit']]

In [7]:
ageGroups = hospitalDf[['HOSP_NIS','AGE']]
ageGroups['AgeBin'] = 0
ageGroups.loc[ageGroups['AGE']<18,'AgeBin'] = 1
ageGroups.loc[ageGroups['AGE']>74,'AgeBin'] = 2
ageGroups = pd.crosstab(ageGroups['HOSP_NIS'],ageGroups['AgeBin']).reset_index()
ageGroups['all'] = ageGroups.iloc[:, 1:].sum(axis=1)
ageGroups['%<18'] = ageGroups[1]/ageGroups['all']*100
ageGroups['%75+'] = ageGroups[2]/ageGroups['all']*100
ageGroups = ageGroups[['HOSP_NIS','%<18','%75+']]

In [8]:
nonWhite = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['RACE']).reset_index()
nonWhite['%NonWhite'] = 100 - (nonWhite[1]/nonWhite.iloc[:, 1:].sum(axis=1) * 100)
nonWhite = nonWhite[['HOSP_NIS','%NonWhite']]

In [9]:
payer = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['PAY1']).reset_index()
payer['%MedicaidUI'] = 100 - ((payer[1]+payer[3])/payer.iloc[:, 1:].sum(axis=1) * 100)
payer = payer[['HOSP_NIS','%MedicaidUI']]

In [10]:
surgeries = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['ORPROC']).reset_index()[['HOSP_NIS',1]]
surgeries.columns = ['HOSP_NIS','SurgicalCases']

In [11]:
avgCharge = hospitalDf[['HOSP_NIS','TOTCHG']].groupby('HOSP_NIS').mean().reset_index()
avgCharge.columns = ['HOSP_NIS','AvgCharge']

In [12]:
transOut = pd.crosstab(hospitalDf['HOSP_NIS'],hospitalDf['TRAN_OUT']).reset_index()
transOut['all'] = transOut.iloc[:, 1:].sum(axis=1)
died = hospitalDf[['HOSP_NIS','DIED']].groupby('HOSP_NIS').sum().reset_index()
routineDischarge = transOut[['HOSP_NIS',0,'all']].merge(died)
routineDischarge['%RoutineDischarge'] = ((routineDischarge[0] - routineDischarge['DIED']) /routineDischarge['all']) * 100
routineDischarge = routineDischarge[['HOSP_NIS','%RoutineDischarge']]

In [13]:
sevDf = pd.read_pickle('hcupSeverity.pkl')
hospitalSev = sevDf.loc[sevDf['HOSP_NIS'].isin(hospitalList)]
hospitalSev = hospitalSev.sort_values('HOSP_NIS')

In [14]:
comorbidities = hospitalSev[['HOSP_NIS','CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF', 'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG', 'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 
        'CM_HYPOTHY', 'CM_LIVER','CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE', 'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL', 'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS']]
comorbidities['AvgComorbidities'] = comorbidities.iloc[:, 1:].sum(axis=1)
comorbidities = comorbidities[['HOSP_NIS','AvgComorbidities']].groupby('HOSP_NIS').mean().reset_index()

In [15]:
coreDf = caseCount.merge(transfers).merge(ed).merge(ageGroups).merge(nonWhite).merge(payer).merge(surgeries).merge(avgCharge).merge(routineDischarge).merge(comorbidities)
coreDf.shape[0]==len(hospitalList)

True

In [16]:
region = hospitalDf[['HOSP_NIS','HOSP_DIVISION']].drop_duplicates()
region['Region'] = region['HOSP_DIVISION'].replace({1:'NewEngland', 2:'MiddleAtlantic', 3:'EastNorthCentral', 4:'WestNorthCentral', 5:'SouthAtlantic', 6:'EastSouthCentral', 
                                 7:'WestSouthCentral', 8:'Mountain', 9:'Pacific'})
region = region[['HOSP_NIS','Region']]

In [17]:
area = hospitalDf[['HOSP_NIS','PL_NCHS2006']].groupby(['HOSP_NIS', 'PL_NCHS2006']).size().to_frame().reset_index().sort_values(['HOSP_NIS', 0],ascending=False).groupby('HOSP_NIS').first().reset_index()
area = area[['HOSP_NIS','PL_NCHS2006']]
area.columns = ['HOSP_NIS','Area']
area['Area'] = area['Area'].replace({1:'BigMetro', 2 :'ExburbBigMetro', 3: 'MediumMetro', 4:'SmallMetro', 5:'Micro', 6:'Rural', -99:'Unknown'})

In [18]:
categorical = pd.get_dummies(region.merge(area))

In [19]:
categorical.shape[0] == len(hospitalList)

True

In [20]:
coreDf.to_excel('continuousFeatures.xlsx', index=False)
categorical.to_excel('categorical.xlsx', index=False)

In [21]:
coreDf.head()

,HOSP_NIS,Cases,TransfersIn,EmergencyAdmit,%<18,%75+,%NonWhite,%MedicaidUI,SurgicalCases,AvgCharge,%RoutineDischarge,AvgComorbidities
0,60387,27912,5380,19647,11.439524,22.269991,28.536113,27.099455,7908,21383.204428,81.011751,2.698302
1,60410,26074,4556,12623,25.017259,8.000307,16.967094,43.518447,9380,48483.244880,86.695559,1.868413
2,60198,20807,3548,8533,26.885183,7.819484,25.332821,37.434517,7849,23172.307252,89.705388,1.621185
3,10116,14222,1268,8996,13.908030,22.725355,31.162987,27.710589,4506,35869.701519,78.659823,2.202011
4,10076,13929,560,7062,21.810611,15.715414,35.350707,31.481083,4496,57602.102448,83.918444,1.855912


In [22]:
categorical.head()

,HOSP_NIS,Region_EastNorthCentral,Region_EastSouthCentral,Region_MiddleAtlantic,Region_Mountain,Region_NewEngland,Region_Pacific,Region_SouthAtlantic,Region_WestNorthCentral,Region_WestSouthCentral,Area_BigMetro,Area_ExburbBigMetro,Area_MediumMetro,Area_Micro,Area_Rural,Area_SmallMetro,Area_Unknown
0,10002,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,10004,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,10007,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,10008,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,10009,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
